<a href="https://colab.research.google.com/github/gdeotale/E4P2/blob/master/Assignment9/%20Neural_word_embeddings_and_sentiment_analysis_ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [2]:
generate_bigrams(['This', 'film', 'is', 'terrible'])

['This', 'film', 'is', 'terrible', 'is terrible', 'film is', 'This film']

In [3]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype = torch.float)

In [4]:
import random

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 10.5MB/s]


In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:29, 2.21MB/s]                           
100%|█████████▉| 399054/400000 [00:23<00:00, 17454.61it/s]

In [6]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [7]:
print(vars(train_data.examples[0]))

{'text': ['DR', '.', 'SEUSS', "'", 'HOW', 'THE', 'GRINCH', 'STOLE', 'CHRISTMAS', '/', '(', '2000', ')', '*', '*', '(', 'out', 'of', 'four)<br', '/><br', '/>If', 'you', 'desire', 'to', 'see', 'a', 'holiday', 'movie', 'that', 'will', 'inspire', 'your', 'seasonal', 'spirits', 'and', 'continue', 'the', 'traditional', 'Dr.', 'Seuss', 'classic', 'fable', ',', 'do', "n't", 'see', '"', 'Dr.', 'Seuss', "'", 'How', 'the', 'Grinch', 'Stole', 'Christmas', '.', '"', 'If', 'you', 'are', 'old', 'enough', 'to', 'read', 'this', 'review', ',', 'then', 'you', 'are', 'probably', 'too', 'old', 'to', 'get', 'any', 'kind', 'of', 'enjoyment', 'out', 'of', 'this', 'motion', 'picture', '.', 'It', 'contains', 'lots', 'of', 'colors', ',', 'creative', 'production', 'design', 'and', 'imaginative', 'set', 'and', 'costume', 'construction', ',', 'joyous', 'load', 'noises', ',', 'and', 'the', 'characters', 'are', 'made', 'up', 'to', 'look', 'like', 'the', 'actual', 'inhabitants', 'of', 'the', 'fictional', 'village', 'W

In [15]:
print(TEXT.vocab.itos)
print(LABEL.vocab.stoi)

print(TEXT.vocab.stoi)

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']
defaultdict(<function _default_unk_index at 0x7f99f892f510>, {'neg': 0, 'pos': 1})
defaultdict(<function _default_unk_index at 0x7f99f892f510>, {'<unk>': 0, '<pad>': 1, 'the': 2, ',': 3, '.': 4, 'a': 5, 'and': 6, 'of': 7, 'to': 8, 'is': 9, 'in': 10, 'I': 11, 'it': 12, 'that': 13, '"': 14, "'s": 15, 'this': 16, '-': 17, '/><br': 18, 'was': 19, 'as': 20, 'with': 21, 'movie': 22, 'for': 23, 'film': 24, 'The': 25, 'but': 26, "n't": 27, '(': 28, 'on': 29, ')': 30, 'you': 31, 'are': 32, 'not': 33, 'have': 34, 'his': 35, 'be': 36, 'he': 37, 'one': 38, '!': 39, 'by': 40, 'at': 41, 'all': 42, 'an': 43, 'who': 44, 'from': 45, 'like': 46, 'they': 47, 'so': 48, 'her': 49, 'or': 50, 'about': 51, 'has': 52, "'": 53, 'out': 54, 'just': 55, 'It': 56, 'of the': 57, 'do': 58, '?': 59, 'good': 60, 'some': 61, 'more': 62, '. The': 63, 'very': 64, 'would': 65, 'up': 66, 'what': 67, 'This': 68, 'in the': 69, ', and': 70, 'there': 71, 'time': 

In [8]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

In [10]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [11]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [12]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.1172e-01, -4.9659e-01,  1.6307e-01,  ...,  1.2647e+00,
         -2.7527e-01, -1.3254e-01],
        [-8.5549e-01, -7.2081e-01,  1.3755e+00,  ...,  8.2522e-02,
         -1.1314e+00,  3.9972e-01],
        [-3.8194e-02, -2.4487e-01,  7.2812e-01,  ..., -1.4590e-01,
          8.2780e-01,  2.7062e-01],
        ...,
        [ 2.0115e-01, -1.9684e-01,  6.3546e-01,  ..., -4.6408e-01,
         -5.1683e-01, -3.3123e-01],
        [ 3.0807e-01,  3.8048e-01,  2.4592e-01,  ...,  4.3605e-01,
          1.2803e+00,  1.7042e+00],
        [-4.3279e-01, -8.8524e-06, -4.0274e-02,  ..., -2.3125e-01,
         -1.3754e-01, -2.3578e-02]])

In [13]:

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [17]:

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [18]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [19]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.688 | Train Acc: 56.85%
	 Val. Loss: 0.633 |  Val. Acc: 73.18%
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.649 | Train Acc: 72.07%
	 Val. Loss: 0.496 |  Val. Acc: 77.74%
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.574 | Train Acc: 80.02%
	 Val. Loss: 0.410 |  Val. Acc: 81.71%
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 0.498 | Train Acc: 84.32%
	 Val. Loss: 0.373 |  Val. Acc: 84.60%
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 0.433 | Train Acc: 87.09%
	 Val. Loss: 0.364 |  Val. Acc: 86.49%


In [24]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.379 | Test Acc: 85.52%


In [25]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [26]:
predict_sentiment(model, "This film is not good")

0.98976069688797

In [27]:
predict_sentiment(model, "This film is not great")

1.0

In [28]:
predict_sentiment(model, "This film is good")

1.0

In [29]:
predict_sentiment(model, "This film is great")

1.0

In [30]:
predict_sentiment(model, "This film is pathetic")

0.02516781911253929